### Домашнее задание

### Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [1]:
import requests

In [2]:
class Rate:
    def __init__(self, format_='value'):
        self.format = format_
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

#### 1 вариант:

In [3]:
def name_max_rates():
    # извлекаем данные из файла в переменную response
    result = {}
    response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()['Valute']
    # проходим циклом по словарю
    for currency in response:
        # записываем словарь с новой структурой имя валюты (поле ‘Name’), значение номинала валюты
        result[response[currency]['Name']] =  response[currency]['Value']/response[currency]['Nominal']
    # выделяем из словаря тьюпл название валюты - значение для максимального значения курса
    max_value = max(result.items(), key=lambda x: x[1])   
    return max_value[0]

In [4]:
name_max_rates()

'СДР (специальные права заимствования)'

#### 2 вариант:
использовать класс Rate

In [5]:
class Max_rate(Rate):
    def __init__(self):
        super().__init__(format_='full')

    def currency_name(self, currency):
        """Получение имени валюты"""
        return self.make_format(currency)['Name']
    
    def currency_nominal_value(self, currency):
        """Получение курса валюты вне зависимости от номинала"""
        return self.make_format(currency)['Value']/self.make_format(currency)['Nominal']
    
    def new_data(self):
        """получение нового словаря имя валюты - курс"""
        result = {}
        for currency in self.exchange_rates():
            result[self.currency_name(currency)] = self.currency_nominal_value(currency)
        return result
    
    def max_value(self):
        """получение назнания валюты с максимальным значением курса"""
        return max(self.new_data().items(), key=lambda x: x[1])[0]

In [6]:
m  = Max_rate()

In [7]:
m.currency_nominal_value('JPY')

0.714859

In [8]:
m.new_data()

{'Австралийский доллар': 55.3553,
 'Азербайджанский манат': 44.7113,
 'Фунт стерлингов Соединенного королевства': 99.7034,
 'Армянских драмов': 0.155659,
 'Белорусский рубль': 28.8039,
 'Болгарский лев': 45.8945,
 'Бразильский реал': 14.3335,
 'Венгерских форинтов': 0.249465,
 'Гонконгских долларов': 9.80137,
 'Датская крона': 12.0638,
 'Доллар США': 75.9645,
 'Евро': 89.7369,
 'Индийских рупий': 1.03217,
 'Казахстанских тенге': 0.178997,
 'Канадский доллар': 57.8381,
 'Киргизских сомов': 0.96454,
 'Китайский юань': 11.1183,
 'Молдавских леев': 4.57067,
 'Норвежских крон': 8.476569999999999,
 'Польский злотый': 20.2114,
 'Румынский лей': 18.4923,
 'СДР (специальные права заимствования)': 107.5338,
 'Сингапурский доллар': 55.5499,
 'Таджикских сомони': 7.360899999999999,
 'Турецкая лира': 10.1536,
 'Новый туркменский манат': 21.7352,
 'Узбекских сумов': 0.007398180000000001,
 'Украинских гривен': 2.73204,
 'Чешских крон': 3.3865900000000004,
 'Шведских крон': 8.65692,
 'Швейцарский фран

In [9]:
m.max_value()

'СДР (специальные права заимствования)'

### Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [62]:
class Diff_rate(Rate):
    def __init__(self, format_, diff = True):
        super().__init__()
        self.format = format_
        self.diff = diff

    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        или изменение курса по сравнению в прошлым значением
        """
        response = self.exchange_rates()

        if currency in response:
            if self.format == 'full':
                return response[currency]

            if self.format == 'value' and self.diff:
                return round(response[currency]['Value'] - response[currency]['Previous'], 2)     

In [63]:
d = Diff_rate(format_='full')

In [64]:
d.usd()

{'ID': 'R01235',
 'NumCode': '840',
 'CharCode': 'USD',
 'Nominal': 1,
 'Name': 'Доллар США',
 'Value': 75.9645,
 'Previous': 75.591}

In [65]:
df= Diff_rate(format_='value')

In [66]:
df.usd()

0.37

### Задание 3

Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла“). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять). Выполните проверку для 20 аккредитаций дизайнера Елены.

In [70]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [76]:
class Designer(Employee):
    def __init__(self, name, seniority, awards):
        super().__init__(name, seniority)
        self.awards = awards

    def check_awards(self):
        self.seniority += 2 * self.awards

    def check_if_it_is_time_for_upgrade(self):
        # по условию увеличиваем аккредитации за имеющиеся международные премии при выходе на работу
        if self.seniority == 0:
            self.check_awards()
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1

        # условие повышения сотрудника из презентации
        if self.seniority % 7 == 0:
            self.grade_up()

        # публикация результатов
        return self.publish_grade()

In [77]:
elena = Designer('Елена', seniority=0, awards=2)
for i in range(20):
    elena.check_if_it_is_time_for_upgrade()

Елена 1
Елена 1
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 4
Елена 4
Елена 4
Елена 4
